In [1]:
import os
import re
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tqdm import tqdm
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import win32api,win32con

# 模板、ptsku、输出文件路径
template_path = './模板.xlsx'
ptsku_path = erp_data_path = list(os.walk('./数据源'))[0][0] + '/' + list(os.walk('./数据源'))[0][2][0]
output_path = './查重结果.xlsx'
# 查重类型、品类全局变量
choose = None
category = None
# 创建表
wb = Workbook()
ws = wb.active
ws.title = "查重结果"

In [2]:
def on_option_click(option): # 选择一三选项后返回序号
    global choose
    choose = option
    root.destroy()
    return choose

def show_category_input():  
    # 创建第二窗口填写品类
    input_window = tk.Toplevel(root)
    input_window.title("品类选择（空白默认全品类）")
    input_window.geometry(f"{window_width}x{window_height}+{x}+{y}")
    # 窗口样式
    input_window.configure(bg="#b3e5fc")
    frame = ttk.Frame(input_window, padding="20", relief="flat")
    frame.grid(row=0, column=0, padx=20, pady=20, sticky=(tk.W, tk.E, tk.N, tk.S))
    # 标签
    ttk.Label(frame, text="请输入品类简称（若有多个以英文分号隔开）\n例如：ACC;BDA;CMPS", font=("微软雅黑", 10), 
              foreground="#0277bd", background="#b3e5fc").grid(row=0, column=0, padx=5, pady=10, sticky=tk.W)
    # 输入框
    category_entry = ttk.Entry(frame, width=40)
    category_entry.grid(row=1, column=0, padx=5, pady=10, sticky=tk.W)
    
    def on_close(): #获取填写品类
        global category
        category = category_entry.get().strip()
        input_window.destroy()
        root.destroy()
        return category
    
    # 确认和返回
    button_frame = ttk.Frame(frame)
    button_frame.grid(row=2, column=0, pady=10)
    tk.Button(button_frame, text="确认", command=on_close, relief="flat", bg="#0277bd", fg="#ffffff",
              font=("微软雅黑", 10, "bold")).grid(row=0, column=0, padx=10)
    tk.Button(button_frame, text="返回", command=input_window.destroy, relief="flat", bg="#4fc3f7", fg="#ffffff",
              font=("微软雅黑", 10, "bold")).grid(row=0, column=1, padx=10)

def create_button(parent, text, command): #主界面选项样式
    button = tk.Button(parent, text=text, command=command, relief="flat", width=40,
                       bg="#0288d1", fg="#ffffff", font=("微软雅黑", 10, "bold"))
    button.bind("<Enter>", lambda e: button.configure(bg="#4fc3f7"))
    button.bind("<Leave>", lambda e: button.configure(bg="#0288d1"))
    button.grid(padx=45, pady=15)
    return button

# 主界面
root = tk.Tk()
root.title("查重类型")
root.configure(bg="#e1f5fe")
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = 500
window_height = 350
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2
root.geometry(f"{window_width}x{window_height}+{x}+{y}")
main_frame = ttk.Frame(root, padding="20", relief="flat")
main_frame.grid(row=0, column=0, padx=20, pady=20, sticky=(tk.W, tk.E, tk.N, tk.S))

# 主界面选项
create_button(main_frame, "1.表内查重", lambda: on_option_click(1)).grid(row=0, column=0)
create_button(main_frame, "2.系统查重（已知品类，不使用模板PTSKU数据）", show_category_input).grid(row=1, column=0)
create_button(main_frame, "3.自定义查重（使用模板PTSKU，匹配已开发信息）", lambda: on_option_click(3)).grid(row=2, column=0)
create_button(main_frame, "4.自定义查重（使用模板PTSKU，不匹配已开发信息）", lambda: on_option_click(4)).grid(row=3, column=0)

root.mainloop()

In [3]:
# 定义oe清洗函数
def normalise_oe(oe):
    oe = str(oe).strip() #清空前后空字符串
    oe = re.sub(r'[\u4e00-\u9fa5]+[a-zA-Z]*','',oe) #去除汉字及后面所有字母
    oe = re.sub('\（','(',oe) #将所有中文"（"符号替换为英文"("
    oe = re.sub('\）',')',oe) #将所有中文"）"符号替换为英文")"
    oe = re.sub('\\([^)]*\\)','',oe)  #删除所有"()"及其内部的字符（因为通常不是OE号）
    oe = re.sub('\\*+', '', oe) #删除所有的"*"

    oe = re.sub('  +', ';', oe) #将两个以上的空格替换为';'
    oe = re.sub(' ', '', oe) #删除单空格
    oe = re.sub('/+', ';', oe) #'/'改为';'
    oe = re.sub('\n', ';', oe) #将换行符替换为';'
    oe = re.sub('[；，,、+:：]', ';', oe) #将[；，,、:：]替换为';'
    oe = re.sub('\u200e', '', oe) #清除不可见字符
    oe = re.sub('\\-', '', oe) #清除"-"
    
    list1 = [i for i in str(oe).split(';') if i != '' and i.isalpha() == False and not re.fullmatch(r'[a-zA-Z]+', i) == True] #去除空值及纯字母
    list1 = [re.sub(r'\.+', '', i) for i in list1] # 去除"."

    # # 对于每个oe中的'/', 如果是以'/'加一到三个字母或数字结尾则去除'/',反之替换为';'
    # list1 = [re.sub(r'/+', '' , i) if len(re.findall(r'/[0-9a-zA-Z]$|/[0-9a-zA-Z]{2}$|/[0-9a-zA-Z]{3}$', i)) != 0 else re.sub(r'/+', ';' , i) for i in list1]
    
    list1 = [i for i in list1 if i != '' and i.isalpha() == False]  #去除空值
    list1 = [re.sub(r'\r','',i) for i in list1]
    list1 = [i for i in list1 if len(i)>2] # 去除一些1位数和2位数字符串
    list1 = [oe.upper() for oe in list1] # 将OE中的字母全部转换为大写
    list2 = list(set(list1)) #去重
    list2.sort(key = list1.index)# 去重后保持OE号原来的顺序
    oe=';'.join(list2).replace(';;', ';')
    try:
        if oe[0] == ';':
            oe = oe[1:]
        elif oe[-1] == ';':
            oe = oe[:-1]
    except:
        pass
    return str(oe.upper())

In [4]:
def count_repeat_number(df):
    list_repeat_count = []
    list_repeat_number = df['完整表内重复辨识号'].tolist()
    for i in list_repeat_number:
        if i == '':
            list_repeat_count.append('-')
        else:
            list_repeat_count.append(int(i.count(';')) + 1)
    df['计数-完整表内重复辨识号'] = list_repeat_count
    return df

In [5]:
# 需查重数据清洗
fac_data = pd.read_excel(template_path, sheet_name='需查重数据', dtype=str).fillna('')
fac_data['清理后OE'] = [normalise_oe(fac_data.loc[i, 'OE']) for i in range(len(fac_data))]

In [6]:
def Duplicate_check_table(fac_data):
    # 剔除无OE的数据，将oe列表的每个元素转换为行
    clean_fac_data = fac_data[fac_data['清理后OE']!='']
    new_fac_data = clean_fac_data[['辨识号','清理后OE']].copy()
    new_fac_data['清理后OE'] = new_fac_data['清理后OE'].str.split(';')
    new_fac_data = new_fac_data.explode('清理后OE')
    
    # 匹配重复项，聚合得到重复辨识号
    merge_fac_data = pd.merge(new_fac_data, new_fac_data, on='清理后OE', how='left')
    group_merge_fac_data = merge_fac_data.groupby(['清理后OE'], as_index=False, sort=False).agg({'辨识号_y':';'.join})
    
    # 匹配进按oe拆分的数据，筛选不与自身重复的值，得到重复辨识号的值，再按辨识号聚合
    repeat_fac_data = pd.merge(new_fac_data, group_merge_fac_data, on='清理后OE', how='left').fillna('')
    fac_repeat_data = repeat_fac_data[repeat_fac_data['辨识号'] != repeat_fac_data['辨识号_y']]
    group_fac_repeat_data = fac_repeat_data.groupby(['辨识号'], as_index=False, sort=False).agg({'清理后OE':';'.join, '辨识号_y':';'.join})
    
    # 得到表内重复OE号和辨识号并去重，保留列表格式
    group_fac_repeat_data.rename(columns={'清理后OE':'表内重复OE号', '辨识号_y':'表内重复辨识号'},inplace=True)
    group_fac_repeat_data['表内重复OE号'] = group_fac_repeat_data['表内重复OE号'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))
    group_fac_repeat_data['表内重复辨识号'] = group_fac_repeat_data['表内重复辨识号'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))
    
    # 按需查重数据辨识号顺序给单元格数据排序
    sort_list = [i for i in fac_data['辨识号']]
    group_fac_repeat_data['表内重复辨识号'] = group_fac_repeat_data['表内重复辨识号'].apply(lambda x:sorted(x,key=sort_list.index))
    
    # 取表内重复辨识号列表重新生成dataframe
    repeat_sku_data = group_fac_repeat_data[['表内重复辨识号']].copy()
    df_alternate = pd.DataFrame(repeat_sku_data['表内重复辨识号'].tolist()).fillna('')

    # 取所有有交集数据的并集
    count = 0
    while True:
        print('# = = = = = = = ' + str(count) + ' = = = = = = = = = =')
        list_row = []
        for i in tqdm(range(len(df_alternate)), desc='Processing', ncols=100):
        # for i in range(len(df_alternate)):
            list_sku = list(set(df_alternate.loc[i].tolist()))
            if '' in list_sku:
                list_sku.remove('')
            find = False
            for sku in list_sku:
                for j in range(len(list_row)):
                    if sku in list_row[j]:
                        list_row[j] = list(set(list_row[j] + list_sku))
                        find = True
                        break
                if find == True:
                    break
            if find == False:
                list_row.append(list_sku)  
        if len(list_row) == len(df_alternate):
            break
        else:
            df_alternate = pd.DataFrame(list_row).fillna('')
            df_alternate.columns = ['内部重复辨识号'+str(i) for i in range(len(df_alternate.columns))]
            count += 1
    
    # 拆分
    df_alternate['完整辨识号'] = df_alternate.apply(lambda x: sorted([i for i in x if i != ''], key=sort_list.index), axis=1)
    all_repeat_fac_data = df_alternate[['完整辨识号']].copy()
    all_repeat_fac_data['拆分辨识号'] = all_repeat_fac_data['完整辨识号'].copy()
    
    # 转换形成辨识号及对应完整表内重复辨识号
    final_all_repeat_fac_data = all_repeat_fac_data.explode('拆分辨识号')
    final_all_repeat_fac_data.rename(columns={'完整辨识号':'完整表内重复辨识号', '拆分辨识号':'辨识号'},inplace=True)

    try:
        # 匹配进原聚合得到表内重复OE号和辨识号的表，列表数据重新转换为字符串
        group_fac_repeat_data = pd.merge(group_fac_repeat_data, final_all_repeat_fac_data, on='辨识号', how='left')
        group_fac_repeat_data['表内重复辨识号'] = group_fac_repeat_data['表内重复辨识号'].apply(lambda x: ';'.join(x))
        group_fac_repeat_data['完整表内重复辨识号'] = group_fac_repeat_data['完整表内重复辨识号'].apply(lambda x: ';'.join(x))
        
        # 根据完整表内重复辨识号聚合汇总对应oe号，得到完整表内重复OE号
        all_repeat_oe_data = group_fac_repeat_data[['完整表内重复辨识号', '表内重复OE号']].groupby('完整表内重复辨识号', sort=False)['表内重复OE号'].sum()
        all_repeat_oe_data.rename('完整表内重复OE号', inplace=True)
        result_repeat_oe_data = pd.merge(group_fac_repeat_data,all_repeat_oe_data,on='完整表内重复辨识号', how='left')
        result_repeat_oe_data['完整表内重复OE号'] = result_repeat_oe_data['完整表内重复OE号'].apply(lambda x: sorted(list(set(x)), key=x.index))
        result_repeat_oe_data['表内重复OE号'] = result_repeat_oe_data[['完整表内重复OE号', '表内重复OE号']].apply(lambda row:[oe for oe in row['完整表内重复OE号'] if oe in row['表内重复OE号']], axis=1)
    
        # 匹配，OE号列表数据重新转换为字符串
        result = pd.merge(fac_data, result_repeat_oe_data, on='辨识号', how='left').fillna('')
        result['表内重复OE号'] = result['表内重复OE号'].apply(lambda x: ';'.join(x))
        result['完整表内重复OE号'] = result['完整表内重复OE号'].apply(lambda x: ';'.join(x))
        print('存在表内重复')
    except ValueError:
        result = pd.merge(fac_data, group_fac_repeat_data, on='辨识号', how='left').fillna('')
        result['表内重复OE号'] = result['表内重复OE号'].apply(lambda x: ';'.join(x))
        result['完整表内重复辨识号'] = ['' for i in range(len(result))]
        result['完整表内重复OE号'] = ['' for i in range(len(result))]
        print('不存在表内重复')
    result = count_repeat_number(result)
    return result, new_fac_data

In [7]:
def Developed_judgment(sku_data, result, new_fac_data): # 判断是否已开发
    # PTSKU对应OE号清洗
    sku_data['清理后OE'] = [normalise_oe(sku_data.loc[i, 'OE']) for i in range(len(sku_data))]
    
    # 剔除无OE的数据，将清洗后oe列每个元素中每个oe都转换为行
    clean_sku_data = sku_data[sku_data['清理后OE']!='']
    new_sku_data = clean_sku_data[['SKU','清理后OE']].copy()
    new_sku_data['清理后OE'] = new_sku_data['清理后OE'].str.split(';')
    new_sku_data = new_sku_data.explode('清理后OE')
    
    # 与需查重需求进行同意操作的表格进行匹配，合得到重复辨识号（即已开发）
    merge_data = pd.merge(new_fac_data, new_sku_data, on='清理后OE', how='left').fillna('')
    repeat_data = merge_data[merge_data['SKU']!='']
    sku_repeat_data = repeat_data.groupby(['辨识号'], as_index=False, sort=False).agg({'清理后OE':';'.join, 'SKU':';'.join})
    sku_repeat_data.rename(columns={'清理后OE':'系统重复OE', 'SKU':'重复PTSKU'}, inplace=True)
    
    # 得到系统重复OE和重复PTSKU并去重，再与表内查重表格合并
    sku_repeat_data['系统重复OE'] = sku_repeat_data['系统重复OE'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))
    sku_repeat_data['重复PTSKU'] = sku_repeat_data['重复PTSKU'].str.split(';').apply(lambda x: sorted(list((set(x))), key=x.index))
    sku_repeat_data['重复PTSKU'] = sku_repeat_data['重复PTSKU'].apply(lambda x: ';'.join(x))
    sku_repeat_data['系统重复OE'] = sku_repeat_data['系统重复OE'].apply(lambda x: ';'.join(x))
    result = pd.merge(result, sku_repeat_data, on='辨识号', how='left').fillna('')
    
    # 判断是否已开发
    result['是否已开发']=''
    for i in range(len(result)):
        if result.loc[i, '清理后OE'] != '':
            if result.loc[i, '重复PTSKU']!='':
                result.loc[i, '是否已开发'] = '是'
            else:
                if result.loc[i, '表内重复辨识号']!='':
                    result.loc[i, '是否已开发'] = '否，表内重复'
                else:
                    result.loc[i, '是否已开发'] = '否'
        else:
            result.loc[i, '是否已开发'] = '无OE号，待进一步确认'

    print('表内重复数量：', len(result[result['表内重复OE号']!='']))
    print('系统重复数量：', len(result[result['系统重复OE']!='']))
    return result

In [8]:
def PTSKU_message(result, need_ptsku_data2): # 匹配PTSKU信息（工厂及价格）
    # 取重复PTSKU不为空部分，分离为单独的PTSKU
    need_result = result[['辨识号', '重复PTSKU']]
    clean_need_result = need_result[need_result['重复PTSKU']!='']
    new_need_result = clean_need_result[['辨识号','重复PTSKU']].copy()
    split_cols = new_need_result['重复PTSKU'].str.split(';', expand=True)
    
    # 重命名列索引，逐一合并
    split_cols_columns = [f'PTSKU_{int(i)+1}' for i in range(split_cols.shape[1])]
    split_cols.columns = split_cols_columns
    new_need_result = pd.concat([new_need_result[['辨识号']], split_cols], axis=1)
    
    # 匹配供应商及价格
    result_process = new_need_result[['辨识号']]
    for i in split_cols_columns:
        index_i = int(split_cols_columns.index(i))
        index_sku = str(index_i+1)
        process_columns = ['辨识号', i]
        df_process = new_need_result[process_columns]
        df_process = pd.merge(df_process, need_ptsku_data2, left_on=i, right_on='SKU', how='left').fillna('')
        df_process = df_process[['辨识号', i, '供应商简称', '最新价格（含税）', '创建日期']]
        df_process.rename(columns={'供应商简称':'供应商_'+index_sku,
                                   '最新价格（含税）':'采购价_'+index_sku,
                                  '创建日期':'创建日期'+index_sku}, inplace=True)
        result_process = pd.merge(result_process, df_process, on='辨识号', how='left').fillna('')
    
    # 匹配进输入的result中
    result = pd.merge(result, result_process, on='辨识号', how='left').fillna('')
    return result

In [9]:
def result_to_excel(wb, ws, result, output_path): # 创建查重结果表格
    for r_idx, row in enumerate(dataframe_to_rows(result, index=False, header=True), 1):
        ws.append(row)
    ws.column_dimensions['A'].width = 12
    ws.column_dimensions['B'].width = 20
    ws.column_dimensions['C'].width = 20
    ws.column_dimensions['D'].width = 22
    ws.column_dimensions['E'].width = 22
    ws.column_dimensions['F'].width = 22
    ws.column_dimensions['G'].width = 22
    ws.column_dimensions['H'].width = 20
    ws.column_dimensions['I'].width = 20
    ws.column_dimensions['J'].width = 20
    ws.column_dimensions['K'].width = 20
    ws.freeze_panes = "A2"
    font = Font(name='等线', size=9)
    header_font = Font(name='等线', size=10, bold=True)
    fill = PatternFill(start_color="95B3D7", end_color="95B3D7", fill_type="solid")
    alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
        for cell in row:
            cell.font = font
            cell.alignment = alignment
            cell.border = border
    for cell in ws[1]:
        cell.font = header_font
        cell.fill = fill
        cell.alignment = alignment
        cell.border = border
    for row in range(1, ws.max_row + 1):
        ws.row_dimensions[row].height = 18
    wb.save(output_path)

In [10]:
if choose == None and category == None: # 直接关闭窗口
    win32api.MessageBox(0, '无进程，结束', '提示', win32con.MB_OK)
elif choose == 1 and category == None: # 仅表内重复查重
    result, new_fac_data = Duplicate_check_table(fac_data)
    for r_idx, row in enumerate(dataframe_to_rows(result, index=False, header=True), 1):
        ws.append(row)
    ws.column_dimensions['A'].width = 12
    ws.column_dimensions['B'].width = 20
    ws.column_dimensions['C'].width = 20
    ws.column_dimensions['D'].width = 25
    ws.column_dimensions['E'].width = 25
    ws.column_dimensions['F'].width = 30
    ws.column_dimensions['G'].width = 30
    ws.column_dimensions['H'].width = 20
    ws.freeze_panes = "A2"
    font = Font(name='等线', size=9)
    header_font = Font(name='等线', size=10, bold=True)
    fill = PatternFill(start_color="95B3D7", end_color="95B3D7", fill_type="solid")
    alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
        for cell in row:
            cell.font = font
            cell.alignment = alignment
            cell.border = border
    for cell in ws[1]:
        cell.font = header_font
        cell.fill = fill
        cell.alignment = alignment
        cell.border = border
    for row in range(1, ws.max_row + 1):
        ws.row_dimensions[row].height = 18
    wb.save(output_path)
    win32api.MessageBox(0, '表内查重完毕，请点击确定结束进程', '提示', win32con.MB_OK)
elif choose == 3 and category == None: # 使用模板查重，匹配已开发信息
    sku_data = pd.read_excel(template_path, sheet_name='PTSKU', dtype=str).fillna('') # PTSKU数据
    result, new_fac_data = Duplicate_check_table(fac_data)
    result = Developed_judgment(sku_data, result, new_fac_data)
    print('请勿关闭界面，正在匹配已开发信息...')
    ptsku_erp = pd.read_excel(ptsku_path, dtype=str).fillna('') # 数据源数据
    need_ptsku_data2 = ptsku_erp[['SKU', '供应商简称', '最新价格（含税）', '创建日期']]
    result = PTSKU_message(result, need_ptsku_data2)
    result_to_excel(wb, ws, result, output_path)
    win32api.MessageBox(0, '自定义查重完毕，请点击确定结束进程', '提示', win32con.MB_OK)
elif choose == 4 and category == None: # 使用模板查重，不匹配已开发信息
    sku_data = pd.read_excel(template_path, sheet_name='PTSKU', dtype=str).fillna('') # PTSKU数据
    result, new_fac_data = Duplicate_check_table(fac_data)
    result = Developed_judgment(sku_data, result, new_fac_data)
    result_to_excel(wb, ws, result, output_path)
    win32api.MessageBox(0, '自定义查重完毕，请点击确定结束进程', '提示', win32con.MB_OK)
elif choose == None and category == '': # 使用数据源全品类查重
    print('与全品类查重中，耗时较长，请耐心等待...')
    print('请勿关闭界面，正在获取PTSKU源数据...')
    ptsku_erp = pd.read_excel(ptsku_path, dtype=str).fillna('') # 数据源数据
    sku_data = ptsku_erp[['SKU', '查重用OE']].copy() # PTSKU数据
    sku_data.rename(columns={'查重用OE':'OE'}, inplace=True)
    need_ptsku_data2 = ptsku_erp[['SKU', '供应商简称', '最新价格（含税）', '创建日期']]
    result, new_fac_data = Duplicate_check_table(fac_data)
    result = Developed_judgment(sku_data, result, new_fac_data)
    print('正在匹配已开发信息...')
    result = PTSKU_message(result, need_ptsku_data2)
    result_to_excel(wb, ws, result, output_path)
    win32api.MessageBox(0, '与系统全品类查重完毕，请点击确定结束进程', '提示', win32con.MB_OK)
else: # 使用数据源特定品类查重
    print('请勿关闭界面，正在获取PTSKU源数据...')
    ptsku_erp = pd.read_excel(ptsku_path, dtype=str).fillna('') # 数据源数据
    need_ptsku_data1 = ptsku_erp[['SKU', '查重用OE', '品类简称']].copy()
    need_ptsku_data1.rename(columns={'查重用OE':'OE'}, inplace=True)
    list_category = category.split(';')
    # PTSKU数据
    sku_data = pd.DataFrame() 
    for en_category in list_category:
        sku_data_each = need_ptsku_data1.loc[need_ptsku_data1['品类简称'] == en_category]
        sku_data = pd.concat([sku_data, sku_data_each], axis=0)
    sku_data = sku_data[['SKU', 'OE']].reset_index(drop=True)
    need_ptsku_data2 = ptsku_erp[['SKU', '供应商简称', '最新价格（含税）', '创建日期']]
    result, new_fac_data = Duplicate_check_table(fac_data) 
    result = Developed_judgment(sku_data, result, new_fac_data)
    print('正在匹配已开发信息...')
    result = PTSKU_message(result, need_ptsku_data2)
    result_to_excel(wb, ws, result, output_path)
    win32api.MessageBox(0, '{}查重完毕，请点击确定结束进程'.format(category), '提示', win32con.MB_OK)

与全品类查重中，耗时较长，请耐心等待...
请勿关闭界面，正在获取PTSKU源数据...
# = = = = = = = 0 = = = = = = = = = =


Processing: 100%|████████████████████████████████████████████████| 50/50 [00:00<00:00, 25151.74it/s]


# = = = = = = = 1 = = = = = = = = = =


Processing: 100%|████████████████████████████████████████████████| 18/18 [00:00<00:00, 18022.79it/s]


# = = = = = = = 2 = = = = = = = = = =


Processing: 100%|███████████████████████████████████████████████████████████| 17/17 [00:00<?, ?it/s]

存在表内重复


表内重复数量： 50
系统重复数量： 62
正在匹配已开发信息...
